<img src="UNSTA Ingeniería.jpeg" style="height:100px">

# Problema de Asignación

#### Importamos las librerías

In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

## Problema

Tengo una consultora en data science y tengo 5 analistas de datos. Necesito asignar cada uno de ellos a un proyecto en particular de modo de maximizar su eficiencia. Cada analista tiene una eficiencia distinta para cada proyecto debido a sus habilidades, gustos y experiencia. Aquí una tabla.


In [2]:
tabla_eficiencias = pd.DataFrame(data={'Proyecto': [1,2,3,4,5],
                   'Álvaro': [9,2,7,8,6],
                   'Barbi': [6,4,3,7,7],
                   'Carlos': [5,8,7,6,5],
                   'Diana': [7,6,5,9,8],
                   'Elena': [7,5,6,8,9]})
tabla_eficiencias

,Proyecto,Álvaro,Barbi,Carlos,Diana,Elena
0,1,9,6,5,7,7
1,2,2,4,8,6,5
2,3,7,3,7,5,6
3,4,8,7,6,9,8
4,5,6,7,5,8,9


#### Definimos el Solver

Creamos el Solver. Para modelos de programación lineal entera, esta vez usaremos CBC.

In [3]:
solver = pywraplp.Solver.CreateSolver('CBC')

## Variables

#### Definimos las variables

$$x_{ij} \text{:Variable dicotómica (dummy) que equivale 1 si el empleado i está en el proyecto j, de lo contrario 0}$$




Para crear las variables en Python, como son enteras, utilizaremos la función BoolVar, formada en el objeto Solver. Las crearemos con un bucle for.

In [4]:
empleados = tabla_eficiencias.columns[1:].tolist()
cantidad_empleados = len(empleados) # Esto es n

proyectos = tabla_eficiencias['Proyecto'].tolist()
cantidad_proyectos = len(proyectos) # Esto es m

In [5]:
variables = {}
for empleado in empleados:
    for proyecto in proyectos:
        variable = solver.BoolVar(f'X_{empleado}_{proyecto}')
        variables[(empleado, proyecto)] = variable

## Restricciones

Las restricciones son las siguientes:

- Máxima cantidad de empleados por proyecto
$$ \sum_{i=1}^{n} x_{ij} = 1 \qquad \forall j \in \{1, 2, 3, 4, 5\} $$

- Máxima cantidad de proyectos por empleado
$$ \sum_{j=1}^{m} x_{ij} = 1 \qquad \forall i \in \{1, 2, 3, 4, 5\} $$


In [6]:
#Máxima cantidad de empleados por proyecto
for proyecto in proyectos:
    ct = solver.Constraint(1, 1, f'Máxima cantidad de empleados para el proyecto {proyecto}')
    for empleado in empleados:
        ct.SetCoefficient(variables[(empleado, proyecto)], 1)
        

In [7]:
#Máxima cantidad de proyectos por empleado
for empleado in empleados:
    ct = solver.Constraint(1, 1, f'Máxima cantidad de proyectos para el empleado {empleado}')
    for proyecto in proyectos:
        ct.SetCoefficient(variables[(empleado, proyecto)], 1)

## Función Objetivo


\begin{align*}
\text{Maximizar} \sum_{i=1}^{n}\sum_{j=1}^{m} c_{ij} \cdot x_{ij}\\
\end{align*}


In [8]:
# Diccionario de eficiencias para acceder mejor a los valores de cada celda
eficiencias = {}
for empleado in empleados:
    for proyecto in proyectos:
        eficiencias[(empleado, proyecto)] = int(tabla_eficiencias.loc[tabla_eficiencias['Proyecto'] == proyecto, empleado].values[0])

In [9]:
objective = solver.Objective()
for empleado in empleados:
    for proyecto in proyectos:
        objective.SetCoefficient(variables[(empleado, proyecto)], eficiencias[(empleado, proyecto)])
objective.SetMaximization()

## Resumen

In [10]:
print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

 Generated by MPModelProtoExporter
   Name             : 
   Format           : Free
   Constraints      : 10
   Variables        : 25
     Binary         : 25
     Integer        : 0
     Continuous     : 0
Maximize
 Obj: +9 X_Álvaro_1 +2 X_Álvaro_2 +7 X_Álvaro_3 +8 X_Álvaro_4 +6 X_Álvaro_5 +6 X_Barbi_1 +4 X_Barbi_2 +3 X_Barbi_3 +7 X_Barbi_4 +7 X_Barbi_5 +5 X_Carlos_1 +8 X_Carlos_2 +7 X_Carlos_3 +6 X_Carlos_4 +5 X_Carlos_5 +7 X_Diana_1 +6 X_Diana_2 +5 X_Diana_3 +9 X_Diana_4 +8 X_Diana_5 +7 X_Elena_1 +5 X_Elena_2 +6 X_Elena_3 +8 X_Elena_4 +9 X_Elena_5 
Subject to
 Máxima_cantidad_de_empleados_para_el_proyecto_1: +1 X_Álvaro_1 +1 X_Barbi_1 +1 X_Carlos_1 +1 X_Diana_1 +1 X_Elena_1  = 1
 Máxima_cantidad_de_empleados_para_el_proyecto_2: +1 X_Álvaro_2 +1 X_Barbi_2 +1 X_Carlos_2 +1 X_Diana_2 +1 X_Elena_2  = 1
 Máxima_cantidad_de_empleados_para_el_proyecto_3: +1 X_Álvaro_3 +1 X_Barbi_3 +1 X_Carlos_3 +1 X_Diana_3 +1 X_Elena_3  = 1
 Máxima_cantidad_de_empleados_para_el_proyecto_4: +1 X_Álvaro_4 

## Resolver el problema

In [11]:
status = solver.Solve()

## Resultados

In [12]:
if status == pywraplp.Solver.OPTIMAL:
    print('El problema tiene una solución óptima')
else:
    print('No se encontró una solución óptima')

El problema tiene una solución óptima


In [13]:
eficiencia_total = solver.Objective().Value()
print(f'La eficiencia total es {eficiencia_total}')

La eficiencia total es 39.0


In [14]:
# Imprimo las variables con valor a 1 en la solución óptima:
for variable in solver.variables():
    if variable.solution_value() == 1:
        print(variable)

X_Álvaro_1
X_Barbi_5
X_Carlos_2
X_Diana_4
X_Elena_3


In [15]:
tabla_eficiencias

,Proyecto,Álvaro,Barbi,Carlos,Diana,Elena
0,1,9,6,5,7,7
1,2,2,4,8,6,5
2,3,7,3,7,5,6
3,4,8,7,6,9,8
4,5,6,7,5,8,9
